#Extract
Authors: Gustavo FLEURY && Induraj RAMAMURTHY

Project: https://github.com/gustavofleury/Audit_Reports_NLP

#Take the raw['content'] in DF and transform Columns - With ACCENTUATION



In [0]:
#Libraries
import pandas as pd
import unicodedata as ud
import re

#Paths
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
FINAL_FILE_DF = DATASETS_FOLDER + 'DF_AAC_ReportsInColumns_ACCENTUATION_v28.02.14.pkl'

In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import FILES
#List of Files - Combine AAG, AAC and ARG
# lstTypeReport=['AAC','AAG','ARG']
lstTypeReport=['AAC'] # Test
dfFile= pd.DataFrame()
for typeReport in lstTypeReport:
  fileName = DATASETS_FOLDER + "DF_Links_Reports_" + typeReport + ".pkl"
  if dfFile.empty:
    dfFile=pd.read_pickle(fileName)
  else:
    print('passed')
    dfFile=dfFile.append(pd.read_pickle(fileName), ignore_index=True)

#DF of Reports already Parsed
df_file_STR_Report = DATASETS_FOLDER + "DF_Reports_STR.pkl"
dfSTR=pd.read_pickle(df_file_STR_Report)

#DF of list of OrgSub (From Expenses DataBase)
dfOrgSub=pd.read_pickle( DATASETS_FOLDER + "OrgaoSubordinado_Unique.pkl")


In [0]:
#Some Info about Files
print('Files just STR    : ' + str(len(dfSTR)))
print('Files with Columns: ' + str(len(dfFile)))
print('Files with Columns: ' + str(len(dfFile)))



Files just STR    : 10879
Files with Columns: 4019
Files with Columns: 4019


In [0]:
# dfSTR.Content[1]
dfSTR.head(2)
# dfFile.head(20)
# dfSTR['FileName'][ dfFile.Links.str.split('/').apply(lambda x: x[-1]) ].reset_index(drop=True)
# dfFile.Links.str.split('/').apply(lambda x: x[-1])
# dfSTR['FileName']=='13175.pdf'

,FileName,Content,MetaData
0,6443.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"{'Author': 'Heleno Vieira Borges', 'Content-Ty..."
1,13421.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"{'Author': 'Danusa da Matta Duarte Fattori', '..."


In [0]:
#
def cleanReport(report):
  #Normalize, delete excess /N and spaces, Lowercase,
  
  # Remove Accentuation
  # report=ud.normalize('NFKD', report).encode('ASCII','ignore').decode('ASCII').strip().lower()
  # WITH Accentuation
  # report=ud.normalize('NFD', report).strip().lower()
  report=report.strip().lower()
  
  report=report.replace('\n', '')
  report=re.sub(' +', ' ', report)

  #Repeated Elements to Delete (like page headers and footers notes)
  list2Delete=['dinheiro público é da sua conta',
              'www.portaldatransparencia.gov.br',
              'presidência da república', 
              'controladoria-geral da união',
              'secretaria federal de controle interno'
              ]

  for text in list2Delete:
    report=report.replace(text,'')

  report=re.sub(' +', ' ', report)
  report=re.sub(' - ', ' ', report)
  report=re.sub('________________', '_', report)

  return report

#
def findReportType(fileNumber, dfLinks):  
  reportType =  dfLinks['Linha de Atuação'][ dfLinks['Links'].str.contains("/"+fileNumber+".pdf") ].reset_index(drop=True)
  
  return reportType[0]


In [0]:
#
def findHeader(report):
  #GERAL
  startText='(unidade\sauditada:\s|unidade\sauditada\s:\s|relatório\sde\savaliação)'
  # startText='(unidade\sauditada:\s|unidade\sauditada\s:\s)'
  endText='(escopo\sdo\strabalho|sumário\s)'
  # endText='(resultados\sdos\strabalhos|resultado\sdos\sexames)'

  #Relatorio de Fiscalizacao
  startTextRF='(relatório)'
  endTextRF='(2\.\sresultados\sdos\sexames|ii\sresultado\sdos\sexames|ii\sresultados\sdos\sexames)'
  
  #Report ARG (example: 13175)
  startTextARG='(unidade\sauditada:\s|unidade\sauditada\s:\s)'
  endTextARG='(2\.\sresultados\sdos\strabalhos|2\.\sresultado\sdos\strabalhos)'

  if bool(re.search(startText + '(.*?)'+ endText,report)) :
    reportHEADER=re.search(startText + '(.*?)'+ endText,report)[0]
    # reportHEADER=re.findall(startText + '(.*?)'+ endText,report)[0][1]
  
  elif bool(re.search(startTextRF + '(.*?)'+ endTextRF, report)) :
    reportHEADER=re.search(startTextRF + '(.*?)'+ endTextRF, report)[0]
    # reportHEADER=re.findall(startTextRF + '(.*?)'+ endTextRF, report)[0][1]
  
  elif bool(re.search(startTextARG + '(.*?)'+ endTextARG, report)) :
    reportHEADER=re.search(startTextARG + '(.*?)'+ endTextARG, report)[0]
  
  else:
    reportHEADER="ERROR: Header not Found"
  return reportHEADER


In [0]:
#
def findReportNumber(reportHEADER):
  startText="(relatório\sn.:|relatório\sn.\s:|ordem\sde\sserviço:|relatório\sn.|número\sdo\srelatório\s:)"
  if bool(re.search(startText+'\s(\d+)', reportHEADER)) :
    reportNum=re.findall(startText+'\s(\d+)', reportHEADER)[0][1]
  else:
    reportNum="ERROR"
  return reportNum

In [0]:
#
def findPeriod(reportHEADER, reportNum):
  month = "(janeiro|fevereiro|marco|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)"
  monthDict={'janeiro':'1','fevereiro':'2','marco':'3','abril':'4','maio':'5','junho':'6','julho':'7','agosto':'8','setembro':'9','outubro':'10','novembro':'11','dezembro':'12'}

  # no período de 01 de janeiro a 31 de dezembro de 2014
  if  bool(re.search('de\s(\d{2})\sde\s'+month+'\sa\s', reportHEADER)) and bool(re.search('a\s(\d{2})\sde\s'+month+'\sde\s(\d{4})', reportHEADER)) :
    dateBEGIN=re.findall('de\s(\d{2})\sde\s'+month+'\sa\s', reportHEADER)
    dateEND=re.findall('a\s(\d{2})\sde\s'+month+'\sde\s(\d{4})', reportHEADER)
    reportYear=dateEND[0][2]
    reportMesBegin=monthDict[dateBEGIN[0][1]]
    reportMesEnd=monthDict[dateEND[0][1]]
    reportBegin=reportYear+'/'+reportMesBegin
    reportEnd=reportYear+'/'+reportMesEnd

  # no exercício de 2014
  elif bool(re.search('no\sexercício\sde\s(\d{4})', reportHEADER)):
    reportYear=re.findall('no\sexercício\sde\s(\d{4})', reportHEADER)[0]
    reportBegin=reportYear+'/1'
    reportEnd=reportYear+'/12'

  # de 01/01/2014 a 31/12/2014
  elif bool(re.search('de\s(\d{2})/(\d{2})/(\d{4})\sa', reportHEADER)) and bool(re.search('a\s(\d{2})/(\d{2})/(\d{4})', reportHEADER)) :
    dateBEGIN=re.findall('de\s(\d{2})/(\d{2})/(\d{4})\sa', reportHEADER)
    dateEND=re.findall('a\s(\d{2})/(\d{2})/(\d{4})', reportHEADER)
    reportYearBEGIN=dateBEGIN[0][2]
    reportYearEND=dateEND[0][2]
    reportMesBegin=str(int(dateBEGIN[0][1]))
    reportMesEnd=str(int(dateEND[0][1]))
    reportBegin=reportYearBEGIN+'/'+reportMesBegin
    reportEnd=reportYearEND+'/'+reportMesEnd
  
  # Exercício 2018 (Relatório de Avaliação)
  elif "relatório de avaliação" in reportHEADER and bool(re.search('exercício\s(\d{4})', reportHEADER)):
    reportYear=re.findall('exercício\s(\d{4})', reportHEADER)
    reportBegin=str(reportYear[0]) + '/1'
    reportEnd=str(reportYear[0]) +'/12'

  # If not Found, take de last year from date of Report
  elif "ERR" not in reportNum:
    reportBegin= str(int(reportNum[0:4])-1) + '/1'
    reportEnd= str(int(reportNum[0:4])-1)+ '/12'
  
  else:
    reportBegin= 'ERROR'
    reportEnd= 'ERROR'
  return reportBegin, reportEnd

In [0]:
############
def findConst(report, linhaAtuacao):
  constatType="constatação|informação"
  #COLLECT the NUMBER of Constat
  # constat (NumC, TypeC, TitleC, FactC, NextNumC)
  constat=[]
  

  ###############
  # Report ARG
  if linhaAtuacao=='Avaliação dos Resultados da Gestão':
    fountParsePattern=0
    if bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')', report) ):
      fountParsePattern=1
      constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')', report)
      # constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+').*?fato', report)
        
      for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusão|3\.\sconclusão)', report) ):
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusão|3\.\sconclusão)', report)[0] )
          elif bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(recomendações)', report) ):
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(recomendações)', report)[0] )
          else:
            constat=[('ERROR', 'ERROR-LastConst-6', 'ERROR', 'ERROR', 'ERROR')]
        else :
          if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report) ):
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report)[0] )
          elif bool( re.search('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report) ): # Reports without field FATO
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report)[0] )
          else:
            constat=[('ERROR', 'ERROR-beetwenConst-6', 'ERROR', 'ERROR', 'ERROR')]


    if bool( re.search('2\.\sresultados\sdos\strabalhos(.*?3\.\sconclusão)', report) ):
      fountParsePattern=1
      textBeetwen=re.findall('2\.\sresultados\sdos\strabalhos(.*?3\.\sconclusão)', report)

      constatText=textBeetwen[0]
      constatLST= re.findall('\s(\d\.\d+|\d\.\d+\.)\s', constatText)
      # constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)

      if len(constatLST)==0:
        constat.append(('2.', '', '', constatText, ''))
      else:
        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(3\.\sconclusão)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(3\.\sconclusão)', constatText)[0]
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst-5', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-5', 'ERROR', 'ERROR', 'ERROR'))

    elif bool( re.search('\sresultados\sdos\sexames(.*?\sconclusão)', report) ) or bool( re.search('\sresultados\sdos\sexames(.*?\srecomendações)', report) ) :
      fountParsePattern=1
      if bool( re.search('\sresultados\sdos\sexames(.*?\srecomendações)', report) ):
        textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\srecomendações)', report)
      else:
        if bool( re.search('\sresultados\sdos\sexames(.*?\sconclusão)', report) ):
          textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\sconclusão)', report)

      if len(textBeetwen)>=2:
        constatSummary=textBeetwen[0]
        constatText=textBeetwen[1]
        constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatSummary)

        titles=[]
        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendações|conclusão)', constatSummary) ):
              titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendações|conclusão)', constatSummary)[0] )
            else:
              titles.append(('ERROR', 'ERROR-Rel_Aval-LastConst-4', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary) ):
              titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary)[0] )
            else:
              titles.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-4', 'ERROR', 'ERROR', 'ERROR'))


        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText)[0]
              constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst-3', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
              constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-3', 'ERROR', 'ERROR', 'ERROR'))

      elif len(textBeetwen)==1:
        constatText=textBeetwen[0]
        constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)
        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText)[0]
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst-3', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-3', 'ERROR', 'ERROR', 'ERROR'))



    if fountParsePattern==0: 
      constat=[('ERROR', 'ERROR-NoConst', 'ERROR', 'ERROR', 'ERROR')]


  ##############
  #FOR 'RELATORIO DE AVALIACAO' 
  elif "relatorio de avaliacao" in report:
    textBeetwen=re.findall('resultados\sdos\sexames(.*?recomendações)', report)

    if len(textBeetwen)>=2:
      constatSummary=textBeetwen[0]
      constatText=textBeetwen[1]
      constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatSummary)

      titles=[]
      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendações)', constatSummary) ):
            titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendações)', constatSummary)[0] )
          else:
            titles.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary) ):
            titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary)[0] )
          else:
            titles.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-2', 'ERROR', 'ERROR', 'ERROR'))


      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações)', constatText)[0]
            constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
            constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-2', 'ERROR', 'ERROR', 'ERROR'))

    elif len(textBeetwen)==1:
      constatText=textBeetwen[0]
      constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)
      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações)', constatText)[0]
            constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst-1', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
            constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst-1', 'ERROR', 'ERROR', 'ERROR'))
    else: 
      constat=[('ERROR', 'ERROR-NoConst', 'ERROR', 'ERROR', 'ERROR')]
  
  
  ###############
  # NORMAL:
  elif bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')', report) ):
    constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')', report)
    # constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+').*?fato', report)
      
    for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
      if n == (len(constatLST)-1) :
        if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusão|iii\s.\sconclusão)', report) ):
          constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusão|iii\s.\sconclusão)', report)[0] )
        else:
          constat=[('ERROR', 'ERROR-LastConst0', 'ERROR', 'ERROR', 'ERROR')]
      else :
        if   bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report) ):
          constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report)[0] )
        elif bool( re.search('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report) ): # Reports without field FATO
          constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report)[0] )
        else:
          constat=[('ERROR', 'ERROR-beetwenConst0', 'ERROR', 'ERROR', 'ERROR')]



  ##############
  # New Reports
  elif bool( re.search('\sresultados\sdos\sexames(.*?\sconclusão)', report) ) or bool( re.search('\sresultados\sdos\sexames(.*?\srecomendações)', report) ) :
    fountParsePattern=1
    if bool( re.search('\sresultados\sdos\sexames(.*?\sconclusão)', report) ):
      textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\sconclusão)', report)    
    else:
      if bool( re.search('\sresultados\sdos\sexames(.*?\srecomendações)', report) ):
        textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\srecomendações)', report)
      
    if len(textBeetwen)>=2:
      constatSummary=textBeetwen[0]
      # constatText=textBeetwen[1]
      constatText=re.findall(textBeetwen[0][-30:]+'(.*?)$', report)[0]
      # constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatSummary)
      constatLST= re.findall('\s(\d{1,3}\.|\d\.\d{1,3})\s', constatSummary)
      
      # print(constatSummary)
      # print(constatText)
      # print(constatLST)
      
      titles=[]
      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendações|conclusão)', constatSummary) ):
            titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendações|conclusão)', constatSummary)[0] )
          else:
            titles.append(('ERROR', 'ERROR-Rel_Aval-LastConst1', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary) ):
            titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary)[0] )
          else:
            titles.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst1', 'ERROR', 'ERROR', 'ERROR'))


      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText)[0]
            constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst2', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
            constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst2', 'ERROR', 'ERROR', 'ERROR'))

    elif len(textBeetwen)==1:
      constatText=textBeetwen[0]
      constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)
      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendações|conclusão)', constatText)[0]
            constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst3', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
            constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst3', 'ERROR', 'ERROR', 'ERROR'))


  #NO MATCH
  else:
    constat=[('ERROR', 'ERROR-NoConst', 'ERROR', 'ERROR', 'ERROR')]

  #CONCLUSION
  if bool( re.search('(iii\s.|iii|3\.)\s(conclusão)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4}\.)', report) ):    
    constat.append( re.findall('(iii\s.|iii|3\.)\s(conclusão)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4}\.)', report)[0] )
    # constat.append( re.findall('(iii)\s(conclusao)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4})', report)[0] )
    # constat.append( re.findall('(iii)\s(conclusao)()(.*?)(\d{4})', report)[0] )
    # constat.append( re.findall('(iii)\s(conclusao)()(.*?)\s(\d{4})', report)[0] )

  # constat (NumC, TypeC, TitleC, FactC, NextNumC)
  return constat

In [0]:
#
def findOrgSub(reportHEADER):
 
  beginText='(unidade\sauditada:|unidade\sauditada\s:)'
  endText='(código|município|exercício)'

  beginTextRA='(unidade\sexaminada:|unidade\sexaminada\s:)'
  endTextRA='(código|município)'
 
  beginTextRF='(unidade\sexaminada:|unidade\sexaminada\s:)'
  endTextRF='(montante\sde)'

  beginTextRAC='(unidade\sexaminada:|unidade\sexaminada\s:)'
  endTextRAC='(município)'
  #General Case
  if bool( re.search(beginText+'\s(.*?)'+endText, reportHEADER) ) :
      orgSub=re.findall(beginText+'\s(.*?)'+endText, reportHEADER)[0][1]

  elif (bool( re.search(beginTextRA+'\s(.*?)'+endTextRA, reportHEADER) )) & ("relatório de avaliação" in reportHEADER) :
    orgSub=re.findall(beginTextRA+'\s(.*?)'+endTextRA, reportHEADER)[0][1]
  
  elif (bool( re.search(beginTextRF+'\s(.*?)'+endTextRF, reportHEADER) )) & ("relatório de fiscalização" in reportHEADER) :
    orgSub=re.findall(beginTextRF+'\s(.*?)'+endTextRF, reportHEADER)[0][1]
  
  elif (bool( re.search(beginTextRAC+'\s(.*?)'+endTextRAC, reportHEADER) )) & ("relatório de ação de controle" in reportHEADER) :
    orgSub=re.findall(beginTextRAC+'\s(.*?)'+endTextRAC, reportHEADER)[0][1]

  else:
    orgSub='ERROR'

  return orgSub.strip()

#
def findOrgSubCod(orgSub, dfFile):
  orgSubCod=""
  for index, row in dfFile.iterrows():
    orgSubTest=(row['Nome Órgão Subordinado']).decode("utf-8")
    if orgSub.find(orgSubTest) != -1 or orgSubTest.find(orgSub) != -1:
      orgSub=orgSubTest
      orgSubCod=row['Código Órgão Subordinado']
  return orgSubCod

In [0]:
# # fileNumberLST=["11573","12564","13231","13349","13359","13319","13374","12375","8834" ]
# fileNumberLST=['13568']
# DEBUG=1

dfFile['fileNumber']=dfFile['Links'].str.split('/').str[-1].str.split('.').str[0]
fileNumberLST=dfFile['fileNumber']
# fileNumberLST=fileNumberLST[:500]
DEBUG=0

organizeLst=[]
i=0
for fileNumber in fileNumberLST:
  if ( len(dfSTR[dfSTR['FileName']==(fileNumber + ".pdf")])>0 ):
    report = dfSTR['Content'][ dfSTR['FileName']==(fileNumber + ".pdf")  ].reset_index(drop=True)[0]
    report = cleanReport(report)    
    reportHeader = findHeader(report)
    reportNum = findReportNumber(reportHeader)
    reportBegin, reportEnd = findPeriod(reportHeader, reportNum)  
    orgSub = findOrgSub(reportHeader)
    orgSubCod = findOrgSubCod(orgSub, dfOrgSub)
    
    reportType = findReportType(fileNumber, dfFile)
    
    linhaAtuacao = dfFile[ dfFile['Links'].str.contains('/' + fileNumber + '.pdf') ]['Linha de Atuação'].reset_index(drop=True)[0]
    if DEBUG==1: print(linhaAtuacao)
    constat = findConst(report, linhaAtuacao)
       
    if ( i%100 == 0 ):
       print("\n" + str(i), end='' )
    i+=1
    print(".", end='')
  
    if DEBUG==1:
      lstVar=['report', 'reportHeader','reportNum','reportBegin','reportEnd',
              'orgSub','reportType']
      for var1 in lstVar:
        print(var1 + ': ' + str(len(eval(var1))) + ' - ' + eval(var1)[0:10])
      # print(constat[0])
  else:
    print('\nFile Parsed not found: ' + fileNumber )

  #Mount DF  
  for numC, typeC, titleC, textC, nextNumC in constat:
    if len(numC) > 0:
      # labelTypeC = 1 if typeC=='constatacao' else 0
      organizeLst.append([orgSubCod,orgSub,fileNumber,reportNum,reportType,reportBegin,reportEnd,numC,typeC, titleC, textC])
  columns=['OrgSubCod','OrgSub','FileNumber','ReportNumber','ReportType','DateBegin','DateEnd','ConstNumber','ConstTYPE','ConstTitle','ConstText']
  dfReportInColumns = pd.DataFrame(organizeLst, columns=columns)
  if DEBUG==1: display(dfReportInColumns)


0....................................................................................................
100....................................................................................................
200.................................
File Parsed not found: 11862
...................................................................
300....................................................................................................
400....................................................................................................
500....................................................................................................
600....................................................................................................
700....................................................................................................
800....................................................................................................
900................................

In [0]:
textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\sconclusão)', report)
print(textBeetwen[0])
# test='análise das demonstrações contábeis.'
constatText=re.findall(textBeetwen[0][-30:]+'(.*?)$', report)
# constatText=re.findall('\sresultados\sdos\sexames(.*?)\sresultados\sdos\sexames(.*?)$', report)
print(constatText)
len(constatText)

 13 1. análise das demonstrações contábeis. 13 2. divergências entre os dados das demonstrações contábeis oficiais e os constantes no siest. 23 3. resultados da gestão. 24 4. estudos de viabilidade econômico-financeira. 29 5. transferência de tecnologia pelo laboratório lfb para a fabricação de hemoderivados, até a desmobilização em 2016. 36 6. [...] 43 7. falhas no planejamento das formações a serem realizadas pelo transferidor de tecnologia de hemoderivados. 46 8. transferência de tecnologia para fabricação de medicamento recombinante, por meio de parceria para o desenvolvimento produtivo (pdp). 47 9. deficiências no monitoramento de contrato de renovação de licença e suporte de ti. 56 10. quadro de pessoal responsável pela gestão de ti. 58 11. conformidade do rol de responsáveis. 59 12. cumprimento de deliberações exaradas pelo tcu. 59 13. atendimento às recomendações do controle interno. 60 recomendações 61 conclusão
[' 62 anexos 65 i – manifestação da unidade examinada e análise d

1

In [0]:
# dfReportInColumns[ dfReportInColumns['ConstText']=='ERROR' ].head(10)
# dfReportInColumns[ dfReportInColumns['FileNumber']=='13014' ].head(10)
# len(dfReportInColumns[ dfReportInColumns['ConstText']!='ERROR' ])
# dfReportInColumns
# reportHeader
len(dfFile)

4019

In [0]:
#VALIDATE RESULTS
print('Total Reports       : ' + str(len(fileNumberLST)))
print('Total Reports Parsed: ' + str(len(dfReportInColumns.FileNumber.unique())))
print('ERROR ReportNumber  : ' + str(len(dfReportInColumns[ (dfReportInColumns.ReportNumber=='ERROR') ].FileNumber.unique() )))
print('ERROR Date          : ' + str(len(dfReportInColumns[ (dfReportInColumns.DateBegin=='ERROR') ].FileNumber.unique() )))
print('ConstNumber OK      : ' + str(len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ].FileNumber.unique() )))
print('ERROR ConstNumber   : ' + str(len(fileNumberLST) - len(dfReportInColumns[ (dfReportInColumns.ConstNumber=='ERROR') ].FileNumber.unique() )))
print('Number ConstatTexts : ' + str(len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ] )))

print('Objective: > 80 %')
print('Accuracy : ' + str( 100*(1-( (len(fileNumberLST)-len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ].FileNumber.unique()))/len(fileNumberLST)))))


Total Reports       : 4019
Total Reports Parsed: 4003
ERROR ReportNumber  : 869
ERROR Date          : 718
ConstNumber OK      : 3915
ERROR ConstNumber   : 2231
Number ConstatTexts : 35642
Objective: > 80 %
Accuracy : 97.41229161482956


In [0]:
#VALIDATE RESULTS
print('Total Reports       : ' + str(len(fileNumberLST)))
print('Total Reports Parsed: ' + str(len(dfReportInColumns.FileNumber.unique())))
print('ERROR ReportNumber  : ' + str(len(dfReportInColumns[ (dfReportInColumns.ReportNumber=='ERROR') ].FileNumber.unique() )))
print('ERROR Date          : ' + str(len(dfReportInColumns[ (dfReportInColumns.DateBegin=='ERROR') ].FileNumber.unique() )))
print('ConstNumber OK      : ' + str(len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ].FileNumber.unique() )))
print('ERROR ConstNumber   : ' + str(len(fileNumberLST) - len(dfReportInColumns[ (dfReportInColumns.ConstNumber=='ERROR') ].FileNumber.unique() )))
print('Number ConstatTexts : ' + str(len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ] )))

print('Objective: > 80 %')
print('Accuracy : ' + str( 100*(1-( (len(fileNumberLST)-len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ].FileNumber.unique()))/len(fileNumberLST)))))

Total Reports       : 4019
Total Reports Parsed: 4003
ERROR ReportNumber  : 869
ERROR Date          : 718
ConstNumber OK      : 3904
ERROR ConstNumber   : 2161
Number ConstatTexts : 35210
Objective: > 80 %
Accuracy : 97.13859168947499


In [0]:
dfReportInColumns[ (dfReportInColumns.ConstNumber=='ERROR') ].head(50)
# dfReportInColumns.head()
# dfReportInColumns=pd.read_pickle(FINAL_FILE_DF)

,OrgSubCod,OrgSub,FileNumber,ReportNumber,ReportType,DateBegin,DateEnd,ConstNumber,ConstTYPE,ConstTitle,ConstText
121,29208,companhia de pesquisa de recursos minerais,12556,201800909,Auditoria Anual de Contas,2017/1,2017/12,ERROR,ERROR-LastConst0,ERROR,ERROR
138,26201,colegio pedro ii,13444,201900517,Auditoria Anual de Contas,2018/1,2018/12,ERROR,ERROR-LastConst0,ERROR,ERROR
140,26236,universidade federal fluminense,13445,201900524,Auditoria Anual de Contas,2018/1,2018/12,ERROR,ERROR-LastConst0,ERROR,ERROR
142,,fundação universidade federal sergipe,13136,201800616,Auditoria Anual de Contas,2017/1,2017/12,ERROR,ERROR-LastConst0,ERROR,ERROR
180,,sebrae/pa,11972,201601874,Auditoria Anual de Contas,2015/1,2015/12,ERROR,ERROR-LastConst0,ERROR,ERROR
182,,conselho regional de engenharia e agronomia de...,12319,201801004,Auditoria Anual de Contas,2017/1,2017/12,ERROR,ERROR-LastConst0,ERROR,ERROR
195,,fundo constitucional de financiamento do centr...,13271,201801663,Auditoria Anual de Contas,2017/1,2017/12,ERROR,ERROR-NoConst,ERROR,ERROR
253,,banco nacional de desenvolvimento econômico e ...,10889,201701442,Auditoria Anual de Contas,2016/1,2016/12,ERROR,ERROR-LastConst0,ERROR,ERROR
343,,banco da amazonia s/a,12147,201503754,Auditoria Anual de Contas,2013/1,2013/12,ERROR,ERROR-LastConst0,ERROR,ERROR
345,,secretaria nacional de renda de cidadania,10179,ERROR,Auditoria Anual de Contas,ERROR,ERROR,ERROR,ERROR-LastConst0,ERROR,ERROR


In [0]:
dfReportInColumns.ConstText[0]

'ERROR'

In [0]:
#Write DF to disk
dfReportInColumns.to_pickle( FINAL_FILE_DF )
# dfReportInColumns = pd.read_pickle(FINAL_FILE_DF)